# Stream Data Loading - Good vs Bad Examples

This notebook demonstrates **meaningful** vs **trivial** implementations of stream-based data loading in C#.

## What Are Streams?

Streams provide a common interface for reading and writing data from various sources:
- **File streams** - Reading/writing files
- **Network streams** - HTTP, TCP, web services
- **Memory streams** - In-memory data processing
- **Compressed streams** - ZIP, GZIP data
- **Crypto streams** - Encrypted data

### Key Benefits of Streams:
1. **Memory efficiency** - Process large files without loading everything into memory
2. **Resource management** - Proper disposal with `using` statements
3. **Flexibility** - Same interface for different data sources
4. **Async support** - Non-blocking operations for better performance
5. **Buffering** - Optimized read/write operations

### ✅ Good Examples - Meaningful Stream Usage

In [ ]:
// ✅ GOOD: File reading with proper resource management
public class FileDataLoader
{
    // Reading CSV data with streams
    public static async Task<List<Dictionary<string, string>>> LoadCsvDataAsync(string filePath)
    {
        var records = new List<Dictionary<string, string>>();
        
        using var fileStream = new FileStream(filePath, FileMode.Open, FileAccess.Read);
        using var reader = new StreamReader(fileStream);
        
        // Read header
        var headerLine = await reader.ReadLineAsync();
        if (headerLine == null) return records;
        
        var headers = headerLine.Split(',').Select(h => h.Trim()).ToArray();
        
        // Read data rows
        string line;
        while ((line = await reader.ReadLineAsync()) != null)
        {
            var values = line.Split(',').Select(v => v.Trim()).ToArray();
            var record = new Dictionary<string, string>();
            
            for (int i = 0; i < Math.Min(headers.Length, values.Length); i++)
            {
                record[headers[i]] = values[i];
            }
            
            records.Add(record);
        }
        
        return records;
    }
    
    // Reading large files line by line to avoid memory issues
    public static async IAsyncEnumerable<string> ReadLargeFileAsync(string filePath)
    {
        using var fileStream = new FileStream(filePath, FileMode.Open, FileAccess.Read);
        using var reader = new StreamReader(fileStream);
        
        string line;
        while ((line = await reader.ReadLineAsync()) != null)
        {
            yield return line;
        }
    }
    
    // Writing data with buffering
    public static async Task WriteDataAsync(string filePath, IEnumerable<string> data)
    {
        using var fileStream = new FileStream(filePath, FileMode.Create, FileAccess.Write);
        using var writer = new StreamWriter(fileStream);
        
        foreach (var item in data)
        {
            await writer.WriteLineAsync(item);
        }
        
        await writer.FlushAsync();
    }
    
    // Copy file with progress reporting
    public static async Task CopyFileWithProgressAsync(string sourcePath, string destinationPath, 
                                                      IProgress<double> progress = null)
    {
        const int bufferSize = 4096;
        var buffer = new byte[bufferSize];
        
        using var sourceStream = new FileStream(sourcePath, FileMode.Open, FileAccess.Read);
        using var destinationStream = new FileStream(destinationPath, FileMode.Create, FileAccess.Write);
        
        long totalBytes = sourceStream.Length;
        long totalBytesRead = 0;
        
        int bytesRead;
        while ((bytesRead = await sourceStream.ReadAsync(buffer, 0, buffer.Length)) > 0)
        {
            await destinationStream.WriteAsync(buffer, 0, bytesRead);
            totalBytesRead += bytesRead;
            
            // Report progress
            progress?.Report((double)totalBytesRead / totalBytes * 100);
        }
    }
}

In [ ]:
// ✅ GOOD: HTTP data loading with streams
public class HttpDataLoader
{
    private static readonly HttpClient _httpClient = new HttpClient();
    
    // Download large files without loading into memory
    public static async Task DownloadFileAsync(string url, string destinationPath, 
                                              IProgress<double> progress = null)
    {
        using var response = await _httpClient.GetAsync(url, HttpCompletionOption.ResponseHeadersRead);
        response.EnsureSuccessStatusCode();
        
        var totalBytes = response.Content.Headers.ContentLength ?? 0;
        
        using var contentStream = await response.Content.ReadAsStreamAsync();
        using var fileStream = new FileStream(destinationPath, FileMode.Create, FileAccess.Write);
        
        var buffer = new byte[4096];
        long totalBytesRead = 0;
        int bytesRead;
        
        while ((bytesRead = await contentStream.ReadAsync(buffer, 0, buffer.Length)) > 0)
        {
            await fileStream.WriteAsync(buffer, 0, bytesRead);
            totalBytesRead += bytesRead;
            
            if (totalBytes > 0)
            {
                progress?.Report((double)totalBytesRead / totalBytes * 100);
            }
        }
    }
    
    // Stream JSON data from API
    public static async Task<T> LoadJsonAsync<T>(string url)
    {
        using var response = await _httpClient.GetAsync(url);
        response.EnsureSuccessStatusCode();
        
        using var stream = await response.Content.ReadAsStreamAsync();
        return await System.Text.Json.JsonSerializer.DeserializeAsync<T>(stream);
    }
    
    // Stream large JSON arrays item by item
    public static async IAsyncEnumerable<T> StreamJsonArrayAsync<T>(string url)
    {
        using var response = await _httpClient.GetAsync(url, HttpCompletionOption.ResponseHeadersRead);
        response.EnsureSuccessStatusCode();
        
        using var stream = await response.Content.ReadAsStreamAsync();
        using var reader = new StreamReader(stream);
        
        var line = await reader.ReadLineAsync(); // Skip opening bracket
        
        while ((line = await reader.ReadLineAsync()) != null)
        {
            if (line.Trim() == "]" || line.Trim() == "}") break;
            
            var cleanLine = line.TrimEnd(',');
            if (!string.IsNullOrWhiteSpace(cleanLine))
            {
                yield return System.Text.Json.JsonSerializer.Deserialize<T>(cleanLine);
            }
        }
    }
    
    // Upload data with streaming
    public static async Task<HttpResponseMessage> UploadStreamAsync(string url, Stream dataStream, 
                                                                   string contentType = "application/octet-stream")
    {
        using var content = new StreamContent(dataStream);
        content.Headers.ContentType = new System.Net.Http.Headers.MediaTypeHeaderValue(contentType);
        
        return await _httpClient.PostAsync(url, content);
    }
}

In [ ]:
// ✅ GOOD: Memory streams for data processing
public class MemoryStreamProcessor
{
    // Convert object to bytes and back
    public static byte[] SerializeToBytes<T>(T obj)
    {
        using var memoryStream = new MemoryStream();
        using var writer = new StreamWriter(memoryStream);
        
        var json = System.Text.Json.JsonSerializer.Serialize(obj);
        writer.Write(json);
        writer.Flush();
        
        return memoryStream.ToArray();
    }
    
    public static T DeserializeFromBytes<T>(byte[] data)
    {
        using var memoryStream = new MemoryStream(data);
        using var reader = new StreamReader(memoryStream);
        
        var json = reader.ReadToEnd();
        return System.Text.Json.JsonSerializer.Deserialize<T>(json);
    }
    
    // Process data in chunks
    public static async Task<List<T>> ProcessDataInChunksAsync<T>(IEnumerable<T> data, 
                                                                 Func<List<T>, Task<List<T>>> processor, 
                                                                 int chunkSize = 1000)
    {
        var results = new List<T>();
        
        using var memoryStream = new MemoryStream();
        var chunk = new List<T>();
        
        foreach (var item in data)
        {
            chunk.Add(item);
            
            if (chunk.Count >= chunkSize)
            {
                var processedChunk = await processor(chunk);
                results.AddRange(processedChunk);
                chunk.Clear();
            }
        }
        
        // Process remaining items
        if (chunk.Count > 0)
        {
            var processedChunk = await processor(chunk);
            results.AddRange(processedChunk);
        }
        
        return results;
    }
    
    // Create in-memory data for testing
    public static Stream CreateTestDataStream(int numberOfRecords)
    {
        var memoryStream = new MemoryStream();
        var writer = new StreamWriter(memoryStream, leaveOpen: true);
        
        // Write CSV header
        writer.WriteLine("Id,Name,Email,CreatedDate");
        
        // Write test data
        for (int i = 1; i <= numberOfRecords; i++)
        {
            writer.WriteLine($"{i},User{i},user{i}@example.com,{DateTime.Now.AddDays(-i):yyyy-MM-dd}");
        }
        
        writer.Flush();
        memoryStream.Position = 0; // Reset to beginning
        
        return memoryStream;
    }
}

In [ ]:
// ✅ GOOD: Compressed stream handling
using System.IO.Compression;

public class CompressionProcessor
{
    // Read data from GZIP compressed file
    public static async Task<string> ReadGzipFileAsync(string gzipFilePath)
    {
        using var fileStream = new FileStream(gzipFilePath, FileMode.Open, FileAccess.Read);
        using var gzipStream = new GZipStream(fileStream, CompressionMode.Decompress);
        using var reader = new StreamReader(gzipStream);
        
        return await reader.ReadToEndAsync();
    }
    
    // Write data to GZIP compressed file
    public static async Task WriteGzipFileAsync(string filePath, string content)
    {
        using var fileStream = new FileStream(filePath, FileMode.Create, FileAccess.Write);
        using var gzipStream = new GZipStream(fileStream, CompressionLevel.Optimal);
        using var writer = new StreamWriter(gzipStream);
        
        await writer.WriteAsync(content);
        await writer.FlushAsync();
    }
    
    // Process ZIP archive entries
    public static async Task<Dictionary<string, string>> ReadZipArchiveAsync(string zipFilePath)
    {
        var files = new Dictionary<string, string>();
        
        using var fileStream = new FileStream(zipFilePath, FileMode.Open, FileAccess.Read);
        using var archive = new ZipArchive(fileStream, ZipArchiveMode.Read);
        
        foreach (var entry in archive.Entries)
        {
            if (!entry.Name.EndsWith(".txt", StringComparison.OrdinalIgnoreCase))
                continue;
                
            using var entryStream = entry.Open();
            using var reader = new StreamReader(entryStream);
            
            var content = await reader.ReadToEndAsync();
            files[entry.Name] = content;
        }
        
        return files;
    }
    
    // Create ZIP archive from multiple streams
    public static async Task CreateZipArchiveAsync(string zipFilePath, 
                                                   Dictionary<string, Stream> files)
    {
        using var fileStream = new FileStream(zipFilePath, FileMode.Create, FileAccess.Write);
        using var archive = new ZipArchive(fileStream, ZipArchiveMode.Create);
        
        foreach (var file in files)
        {
            var entry = archive.CreateEntry(file.Key);
            
            using var entryStream = entry.Open();
            file.Value.Position = 0; // Reset stream position
            await file.Value.CopyToAsync(entryStream);
        }
    }
}

In [ ]:
// ✅ GOOD USAGE: Demonstrate proper stream usage with real scenarios

// Example 1: Processing large CSV file without loading all into memory
async Task ProcessLargeCsvFile()
{
    var testDataStream = MemoryStreamProcessor.CreateTestDataStream(1000);
    
    // Simulate saving to a file
    var tempFile = Path.GetTempFileName();
    
    try
    {
        // Save test data to file
        using (var fileStream = new FileStream(tempFile, FileMode.Create, FileAccess.Write))
        {
            await testDataStream.CopyToAsync(fileStream);
        }
        
        // Now read it back using streams
        var records = await FileDataLoader.LoadCsvDataAsync(tempFile);
        Console.WriteLine($"Loaded {records.Count} records from CSV");
        
        // Display first few records
        foreach (var record in records.Take(3))
        {
            Console.WriteLine($"ID: {record["Id"]}, Name: {record["Name"]}, Email: {record["Email"]}");
        }
    }
    finally
    {
        testDataStream.Dispose();
        if (File.Exists(tempFile))
            File.Delete(tempFile);
    }
}

await ProcessLargeCsvFile();
Console.WriteLine();

In [ ]:
// Example 2: Memory stream data processing
var testObject = new { Name = "John Doe", Age = 30, City = "New York" };

// Serialize to bytes
var bytes = MemoryStreamProcessor.SerializeToBytes(testObject);
Console.WriteLine($"Serialized object to {bytes.Length} bytes");

// Deserialize back
var deserializedObject = MemoryStreamProcessor.DeserializeFromBytes<object>(bytes);
Console.WriteLine($"Deserialized object: {deserializedObject}");
Console.WriteLine();

// Example 3: Stream-based data transformation
var numbers = Enumerable.Range(1, 100);
var processedNumbers = await MemoryStreamProcessor.ProcessDataInChunksAsync(
    numbers,
    async chunk => 
    {
        // Simulate some async processing
        await Task.Delay(10);
        return chunk.Where(n => n % 2 == 0).Select(n => n * 2).ToList();
    },
    chunkSize: 20
);

Console.WriteLine($"Processed {processedNumbers.Count} even numbers (doubled): [{string.Join(", ", processedNumbers.Take(10))}...]");
Console.WriteLine();

In [ ]:
// Example 4: Compression with streams
var testContent = "This is a test content that will be compressed using GZIP. " +
                 "Streams allow us to handle compression efficiently without loading " +
                 "all data into memory at once. This is especially useful for large files.";

var tempGzipFile = Path.ChangeExtension(Path.GetTempFileName(), ".gz");

try
{
    // Compress data
    await CompressionProcessor.WriteGzipFileAsync(tempGzipFile, testContent);
    
    var originalSize = System.Text.Encoding.UTF8.GetBytes(testContent).Length;
    var compressedSize = new FileInfo(tempGzipFile).Length;
    
    Console.WriteLine($"Original size: {originalSize} bytes");
    Console.WriteLine($"Compressed size: {compressedSize} bytes");
    Console.WriteLine($"Compression ratio: {(double)compressedSize / originalSize:P1}");
    
    // Decompress data
    var decompressedContent = await CompressionProcessor.ReadGzipFileAsync(tempGzipFile);
    Console.WriteLine($"Content matches: {testContent == decompressedContent}");
    Console.WriteLine($"Decompressed content preview: {decompressedContent.Substring(0, 50)}...");
}
finally
{
    if (File.Exists(tempGzipFile))
        File.Delete(tempGzipFile);
}
Console.WriteLine();

### ❌ Bad Examples - Poor Stream Usage

In [ ]:
// ❌ BAD: Not using 'using' statements - resource leaks
public class BadFileReader
{
    // ❌ BAD: Resources not properly disposed
    public static string ReadFileBadly(string filePath)
    {
        var fileStream = new FileStream(filePath, FileMode.Open); // Not disposed!
        var reader = new StreamReader(fileStream); // Not disposed!
        
        return reader.ReadToEnd(); // Resources leaked if exception occurs
    }
    
    // ❌ BAD: Manual disposal without exception handling
    public static string ReadFileManualDisposal(string filePath)
    {
        FileStream fileStream = null;
        StreamReader reader = null;
        
        try
        {
            fileStream = new FileStream(filePath, FileMode.Open);
            reader = new StreamReader(fileStream);
            
            return reader.ReadToEnd();
        }
        finally
        {
            reader?.Dispose(); // What if reader creation failed?
            fileStream?.Dispose(); // Verbose and error-prone
        }
    }
    
    // ❌ BAD: Using File.ReadAllText when streams are required
    public static string ReadFileSimple(string filePath)
    {
        // While this works, it doesn't demonstrate stream usage
        // and loads entire file into memory
        return File.ReadAllText(filePath);
    }
    
    // ❌ BAD: Synchronous operations in async context
    public static async Task<string> ReadFileAsyncBadly(string filePath)
    {
        // Using sync methods in async method defeats the purpose
        using var stream = new FileStream(filePath, FileMode.Open);
        using var reader = new StreamReader(stream);
        
        return reader.ReadToEnd(); // Should be ReadToEndAsync()
    }
}

In [ ]:
// ❌ BAD: Poor HTTP stream handling
public class BadHttpLoader
{
    // ❌ BAD: Not disposing HttpClient (though in real apps, reuse HttpClient)
    public static async Task<string> DownloadBadly(string url)
    {
        var client = new HttpClient(); // Not disposed - socket exhaustion!
        var response = await client.GetAsync(url); // Not disposed!
        
        return await response.Content.ReadAsStringAsync();
    }
    
    // ❌ BAD: Loading entire response into memory
    public static async Task DownloadLargeFileBadly(string url, string filePath)
    {
        using var client = new HttpClient();
        
        // This loads entire file into memory first!
        var data = await client.GetByteArrayAsync(url);
        
        await File.WriteAllBytesAsync(filePath, data);
    }
    
    // ❌ BAD: No error handling or timeout
    public static async Task<string> DownloadWithoutErrorHandling(string url)
    {
        using var client = new HttpClient();
        // No timeout, no error handling, no validation
        var response = await client.GetStringAsync(url);
        return response;
    }
    
    // ❌ BAD: Blocking async operations
    public static string DownloadBlocking(string url)
    {
        using var client = new HttpClient();
        
        // Using .Result can cause deadlocks
        return client.GetStringAsync(url).Result;
    }
}

In [ ]:
// ❌ BAD: Inefficient memory usage
public class BadMemoryUsage
{
    // ❌ BAD: Loading entire large file into string
    public static List<string> ProcessLargeFileInefficiently(string filePath)
    {
        // This loads entire file into memory at once
        var content = File.ReadAllText(filePath);
        var lines = content.Split('\n');
        
        // Then processes it - memory usage is doubled!
        return lines.Where(line => !string.IsNullOrWhiteSpace(line))
                   .Select(line => line.Trim())
                   .ToList();
    }
    
    // ❌ BAD: Creating unnecessary byte arrays
    public static async Task CopyFileInefficiently(string source, string destination)
    {
        // Loads entire file into memory
        var data = await File.ReadAllBytesAsync(source);
        await File.WriteAllBytesAsync(destination, data);
        
        // For large files, this wastes memory and can cause OutOfMemoryException
    }
    
    // ❌ BAD: Not using buffering
    public static async Task CopyFileByteByByte(string source, string destination)
    {
        using var sourceStream = new FileStream(source, FileMode.Open);
        using var destStream = new FileStream(destination, FileMode.Create);
        
        int byteValue;
        // Extremely inefficient - one byte at a time!
        while ((byteValue = sourceStream.ReadByte()) != -1)
        {
            destStream.WriteByte((byte)byteValue);
        }
    }
    
    // ❌ BAD: String concatenation in loops
    public static string ProcessTextFileInefficiently(string filePath)
    {
        using var stream = new FileStream(filePath, FileMode.Open);
        using var reader = new StreamReader(stream);
        
        string result = "";
        string line;
        
        // String concatenation creates new string objects each time
        while ((line = reader.ReadLine()) != null)
        {
            result += line.ToUpper() + "\n"; // Very inefficient!
        }
        
        return result;
    }
}

In [ ]:
// ❌ BAD USAGE: Demonstrate the problems with poor stream usage

// These examples show what NOT to do:
Console.WriteLine("❌ Examples of what NOT to do:");
Console.WriteLine("1. Not using 'using' statements - causes resource leaks");
Console.WriteLine("2. Loading entire large files into memory - causes OutOfMemoryException");
Console.WriteLine("3. Using synchronous methods in async contexts - blocks threads");
Console.WriteLine("4. Reading/writing byte by byte - extremely slow");
Console.WriteLine("5. String concatenation in loops - creates many temporary objects");
Console.WriteLine("6. Not disposing HttpClient properly - socket exhaustion");
Console.WriteLine("7. Using .Result on async operations - can cause deadlocks");
Console.WriteLine("8. No error handling or timeouts - unreliable operations");
Console.WriteLine();

// Show memory efficiency difference
Console.WriteLine("Memory usage comparison:");
Console.WriteLine("✅ Stream-based: Constant memory usage regardless of file size");
Console.WriteLine("❌ File.ReadAllText: Memory usage = file size (+ processing overhead)");
Console.WriteLine("❌ Byte-by-byte: Constant memory but extremely slow (system call per byte)");
Console.WriteLine();

// Performance implications
Console.WriteLine("Performance implications:");
Console.WriteLine("✅ Async streams: Non-blocking, scalable");
Console.WriteLine("✅ Buffered operations: Efficient I/O");
Console.WriteLine("❌ Sync in async: Thread pool starvation");
Console.WriteLine("❌ No buffering: Many system calls");
Console.WriteLine("❌ Resource leaks: Memory and handle exhaustion");

## Domain-Specific Stream Examples

In [ ]:
// ✅ GOOD: Real-world application scenarios
public class LogFileProcessor
{
    // Process log files that might be very large
    public static async IAsyncEnumerable<LogEntry> ReadLogEntriesAsync(string logFilePath)
    {
        using var fileStream = new FileStream(logFilePath, FileMode.Open, FileAccess.Read);
        using var reader = new StreamReader(fileStream);
        
        string line;
        while ((line = await reader.ReadLineAsync()) != null)
        {
            if (TryParseLogEntry(line, out var logEntry))
            {
                yield return logEntry;
            }
        }
    }
    
    private static bool TryParseLogEntry(string line, out LogEntry logEntry)
    {
        logEntry = null;
        
        if (string.IsNullOrWhiteSpace(line))
            return false;
            
        var parts = line.Split(' ', 4);
        if (parts.Length < 4)
            return false;
            
        if (DateTime.TryParse($"{parts[0]} {parts[1]}", out var timestamp))
        {
            logEntry = new LogEntry
            {
                Timestamp = timestamp,
                Level = parts[2],
                Message = parts[3]
            };
            return true;
        }
        
        return false;
    }
    
    // Write filtered logs to new file
    public static async Task FilterLogsAsync(string inputPath, string outputPath, 
                                           string minLevel = "ERROR")
    {
        var levelPriority = new Dictionary<string, int>
        {
            ["DEBUG"] = 1,
            ["INFO"] = 2,
            ["WARN"] = 3,
            ["ERROR"] = 4,
            ["FATAL"] = 5
        };
        
        var minPriority = levelPriority.GetValueOrDefault(minLevel, 4);
        
        using var outputStream = new FileStream(outputPath, FileMode.Create, FileAccess.Write);
        using var writer = new StreamWriter(outputStream);
        
        await foreach (var logEntry in ReadLogEntriesAsync(inputPath))
        {
            var entryPriority = levelPriority.GetValueOrDefault(logEntry.Level, 0);
            if (entryPriority >= minPriority)
            {
                await writer.WriteLineAsync($"{logEntry.Timestamp:yyyy-MM-dd HH:mm:ss} {logEntry.Level} {logEntry.Message}");
            }
        }
    }
}

public class LogEntry
{
    public DateTime Timestamp { get; set; }
    public string Level { get; set; }
    public string Message { get; set; }
}

In [ ]:
// ✅ GOOD: Configuration file processing
public class ConfigurationProcessor
{
    // Read configuration with stream validation
    public static async Task<Dictionary<string, string>> LoadConfigAsync(string configPath)
    {
        var config = new Dictionary<string, string>();
        
        if (!File.Exists(configPath))
            return config;
            
        using var fileStream = new FileStream(configPath, FileMode.Open, FileAccess.Read);
        using var reader = new StreamReader(fileStream);
        
        string line;
        int lineNumber = 0;
        
        while ((line = await reader.ReadLineAsync()) != null)
        {
            lineNumber++;
            
            // Skip comments and empty lines
            line = line.Trim();
            if (string.IsNullOrEmpty(line) || line.StartsWith("#"))
                continue;
                
            var equalIndex = line.IndexOf('=');
            if (equalIndex > 0)
            {
                var key = line.Substring(0, equalIndex).Trim();
                var value = line.Substring(equalIndex + 1).Trim();
                
                if (!string.IsNullOrEmpty(key))
                {
                    config[key] = value;
                }
            }
            else
            {
                Console.WriteLine($"Warning: Invalid config line {lineNumber}: {line}");
            }
        }
        
        return config;
    }
    
    // Write configuration with backup
    public static async Task SaveConfigAsync(string configPath, Dictionary<string, string> config)
    {
        // Create backup if file exists
        if (File.Exists(configPath))
        {
            var backupPath = $"{configPath}.backup";
            await CopyFileWithStreamAsync(configPath, backupPath);
        }
        
        using var fileStream = new FileStream(configPath, FileMode.Create, FileAccess.Write);
        using var writer = new StreamWriter(fileStream);
        
        await writer.WriteLineAsync($"# Configuration file generated on {DateTime.Now:yyyy-MM-dd HH:mm:ss}");
        await writer.WriteLineAsync();
        
        foreach (var kvp in config.OrderBy(x => x.Key))
        {
            await writer.WriteLineAsync($"{kvp.Key}={kvp.Value}");
        }
        
        await writer.FlushAsync();
    }
    
    private static async Task CopyFileWithStreamAsync(string source, string destination)
    {
        using var sourceStream = new FileStream(source, FileMode.Open, FileAccess.Read);
        using var destStream = new FileStream(destination, FileMode.Create, FileAccess.Write);
        
        await sourceStream.CopyToAsync(destStream);
    }
}

In [ ]:
// Usage examples for domain-specific stream processing

// Example 1: Create and process a sample log file
var tempLogFile = Path.GetTempFileName();
var filteredLogFile = Path.ChangeExtension(tempLogFile, ".filtered.log");

try
{
    // Create sample log data
    var sampleLogs = new[]
    {
        $"{DateTime.Now.AddHours(-3):yyyy-MM-dd HH:mm:ss} INFO Application started",
        $"{DateTime.Now.AddHours(-2):yyyy-MM-dd HH:mm:ss} DEBUG User login attempt",
        $"{DateTime.Now.AddHours(-1):yyyy-MM-dd HH:mm:ss} WARN Invalid password attempt",
        $"{DateTime.Now.AddMinutes(-30):yyyy-MM-dd HH:mm:ss} ERROR Database connection failed",
        $"{DateTime.Now.AddMinutes(-15):yyyy-MM-dd HH:mm:ss} FATAL System crash detected",
        $"{DateTime.Now:yyyy-MM-dd HH:mm:ss} INFO System recovered"
    };
    
    await File.WriteAllLinesAsync(tempLogFile, sampleLogs);
    
    Console.WriteLine("Sample log entries:");
    await foreach (var logEntry in LogFileProcessor.ReadLogEntriesAsync(tempLogFile))
    {
        Console.WriteLine($"{logEntry.Timestamp:HH:mm:ss} [{logEntry.Level}] {logEntry.Message}");
    }
    
    // Filter to only ERROR and FATAL messages
    await LogFileProcessor.FilterLogsAsync(tempLogFile, filteredLogFile, "ERROR");
    
    Console.WriteLine("\nFiltered log entries (ERROR and above):");
    var filteredContent = await File.ReadAllTextAsync(filteredLogFile);
    Console.WriteLine(filteredContent);
}
finally
{
    File.Delete(tempLogFile);
    File.Delete(filteredLogFile);
}
Console.WriteLine();

In [ ]:
// Example 2: Configuration file processing
var tempConfigFile = Path.ChangeExtension(Path.GetTempFileName(), ".config");

try
{
    // Create sample configuration
    var config = new Dictionary<string, string>
    {
        ["database.host"] = "localhost",
        ["database.port"] = "5432",
        ["database.name"] = "myapp",
        ["app.debug"] = "true",
        ["app.timeout"] = "30",
        ["logging.level"] = "INFO"
    };
    
    // Save configuration
    await ConfigurationProcessor.SaveConfigAsync(tempConfigFile, config);
    
    Console.WriteLine("Saved configuration file:");
    var savedContent = await File.ReadAllTextAsync(tempConfigFile);
    Console.WriteLine(savedContent);
    
    // Load configuration back
    var loadedConfig = await ConfigurationProcessor.LoadConfigAsync(tempConfigFile);
    
    Console.WriteLine("Loaded configuration:");
    foreach (var kvp in loadedConfig)
    {
        Console.WriteLine($"{kvp.Key} = {kvp.Value}");
    }
    
    Console.WriteLine($"\nConfiguration entries loaded: {loadedConfig.Count}");
}
finally
{
    File.Delete(tempConfigFile);
    var backupFile = tempConfigFile + ".backup";
    if (File.Exists(backupFile))
        File.Delete(backupFile);
}

## Summary

### What Makes Stream Usage "Meaningful":

**Perfect Use Cases:**
- **Large files** - Process without loading everything into memory
- **Network operations** - Download/upload with progress and efficiency
- **Data transformation** - Process data as it flows through
- **Compression** - Handle compressed data efficiently
- **Real-time processing** - Log files, streaming data
- **Resource management** - Proper disposal with `using` statements

**Key Benefits:**
1. **Memory efficiency** - Constant memory usage regardless of data size
2. **Performance** - Async operations don't block threads
3. **Resource safety** - Automatic disposal prevents leaks
4. **Scalability** - Handle large datasets without memory constraints
5. **Flexibility** - Same interface for files, network, memory, compression

### When Streams Add Real Value:

**File Operations:**
- Large file processing: log analysis, data imports
- Real-time file monitoring: tail -f equivalent
- File transformations: encoding, formatting

**Network Operations:**
- Large file downloads with progress
- Streaming API responses
- Real-time data feeds

**Data Processing:**
- CSV/JSON processing without memory limits
- Data compression/decompression
- Encryption/decryption pipelines

### Red Flags (Avoid These):

**Don't Use Streams When:**
- **Small files** - `File.ReadAllText()` is simpler for small files
- **Simple operations** - One-time reads of config files
- **Memory isn't a concern** - Processing small datasets

**Anti-Patterns:**
- Not using `using` statements - causes resource leaks
- Loading entire streams into memory - defeats the purpose
- Synchronous operations in async contexts - blocks threads
- Reading/writing byte by byte - extremely inefficient
- Ignoring error handling - unreliable operations
- Using `.Result` on async operations - can cause deadlocks

### Best Practices:

1. **Always use `using` statements** - Ensures proper resource disposal
2. **Prefer async operations** - Don't block threads unnecessarily
3. **Use appropriate buffer sizes** - Balance memory usage and performance
4. **Handle errors gracefully** - Network and file operations can fail
5. **Consider progress reporting** - For long-running operations
6. **Choose the right stream type** - File, Memory, Network, Compression
7. **Validate inputs** - Check file existence, URL validity, etc.
8. **Consider cancellation** - Support for cancellation tokens

### Examples Summary:

**✅ Good Examples Shown:**
- File operations: CSV processing, large file copying, line-by-line reading
- HTTP operations: Large file downloads, JSON streaming, upload operations
- Memory streams: Data serialization, chunk processing, test data generation
- Compression: GZIP files, ZIP archives, data compression pipelines
- Domain-specific: Log file processing, configuration management
- Resource management: Proper `using` statements, async patterns

**❌ Bad Examples Highlighted:**
- Resource leaks: Not disposing streams properly
- Memory inefficiency: Loading entire files unnecessarily
- Performance issues: Sync operations in async contexts
- Poor error handling: No validation or timeout handling
- Blocking operations: Using `.Result` on async calls

These examples demonstrate how streams enable efficient, scalable, and safe data processing when used appropriately. The key is understanding when the benefits of streams (memory efficiency, resource management, async support) outweigh the simplicity of basic file operations.